In [1]:
import functools
import numpy as np
import gymnasium as gym
from gymnasium import error, spaces, utils
from gymnasium.utils import seeding
from gym.spaces import Discrete, MultiDiscrete
import pygame
import time

import os
from pygame.surfarray import array3d
from pygame import display

from pettingzoo import ParallelEnv
from pettingzoo.utils import parallel_to_aec, wrappers

UP = 0
DOWN = 1
RIGHT = 2
LEFT = 3
GRAP = 4
BREAK = 5

MOVES = ["UP", "DOWN", "RIGHT", "LEFT", "GRAP", "BREAK"]
NUM_ITERS = 100
REWARD_MAP = {
    (ROCK, ROCK): (0, 0),
    (ROCK, PAPER): (-1, 1),
    (ROCK, SCISSORS): (1, -1),
    (PAPER, ROCK): (1, -1),
    (PAPER, PAPER): (0, 0),
    (PAPER, SCISSORS): (-1, 1),
    (SCISSORS, ROCK): (-1, 1),
    (SCISSORS, PAPER): (1, -1),
    (SCISSORS, SCISSORS): (0, 0),
}


def env(render_mode=None):
    """
    The env function often wraps the environment in wrappers by default.
    You can find full documentation for these methods
    elsewhere in the developer documentation.
    """
    internal_render_mode = render_mode if render_mode != "ansi" else "human"
    env = raw_env(render_mode=internal_render_mode)
    # This wrapper is only for environments which print results to the terminal
    if render_mode == "ansi":
        env = wrappers.CaptureStdoutWrapper(env)
    # this wrapper helps error handling for discrete action spaces
    env = wrappers.AssertOutOfBoundsWrapper(env)
    # Provides a wide vareity of helpful user errors
    # Strongly recommended
    env = wrappers.OrderEnforcingWrapper(env)
    return env


def raw_env(render_mode=None):
    """
    To support the AEC API, the raw_env() function just uses the from_parallel
    function to convert from a ParallelEnv to an AEC env
    """
    env = UnrailedEnv(render_mode=render_mode)
    env = parallel_to_aec(env)
    return env

NameError: name 'ROCK' is not defined

In [3]:
class UnrailedEnv(ParallelEnv):
    metadata = {"render_modes": ["human"], "name": "unrailed"}

    def __init__(self, render_mode=None):
        """
        The init method takes in environment arguments and should define the following attributes:
        - possible_agents
        - render_mode

        Note: as of v1.18.1, the action_spaces and observation_spaces attributes are deprecated.
        Spaces should be defined in the action_space() and observation_space() methods.
        If these methods are not overridden, spaces will be inferred from self.observation_spaces/action_spaces, raising a warning.

        These attributes should not be changed after initialization.
        """
        if render_mode == "human":
            pygame.init()
        self.PATH = os.path.dirname(os.path.abspath(__file__))
        self.width = 46
        self.heigth = 20
        self.cell_size = 20
        self.population = {
            0: {
                "name": "EMPTY",
                "color":pygame.Color(0, 0, 0)
                },
            1: {
                "name": "GRASS",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/plain.png"), (self.cell_size, self.cell_size))
                },
            2: {
                "name": "TREE",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/tree-a.png"), (self.cell_size, self.cell_size))
                },
            3: {
                "name": "ROCK",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/iron.png"), (self.cell_size, self.cell_size))
                },
            4: {
                "name": "SOLID",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/rock.png"), (self.cell_size, self.cell_size))
                },
            5: {
                "name": "WATHER",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/water.png"), (self.cell_size, self.cell_size))
                },
            6: {
                "name": "HOUSE",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/station-l.png"), (self.cell_size, self.cell_size))
                },
            7: {
                "name": "BRIDGE",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/bridge.png"), (self.cell_size, self.cell_size))
                },
            8: {
                "name": "RAIL",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/rail-rl.png"), (self.cell_size, self.cell_size))
                },
            9:{
                "name": "WOOD",
                "color": pygame.transform.scale(pygame.image.load(self.PATH + "/assets/resource-wood.png"), (self.cell_size, self.cell_size))
            },
            10:{
                "name": "IRON",
                "color": pygame.transform.scale(pygame.image.load(self.PATH + "/assets/resource-iron.png"), (self.cell_size, self.cell_size))
            },
            11:{
                "name": "AXE",
                "color": pygame.transform.scale(pygame.image.load(self.PATH + "/assets/resource-axe.png"), (self.cell_size, self.cell_size))
            },
            12:{
                "name": "PICKAXE",
                "color": pygame.transform.scale(pygame.image.load(self.PATH + "/assets/resource-pickaxe.png"), (self.cell_size, self.cell_size))
            },
            13:{
                "name": "BUCKET",
                "color": pygame.transform.scale(pygame.image.load(self.PATH + "/assets/resource-backet.png"), (self.cell_size, self.cell_size))
            },
            14:{
                "name": "EMPTY_BUCKET",
                "color": pygame.transform.scale(pygame.image.load(self.PATH + "/assets/resource-backet-empty.png"), (self.cell_size, self.cell_size))
            },
            "PLAYER0": {
                "name": "PLAYER0",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/face-1.png"), (self.cell_size, self.cell_size))
                },
            "PLAYER1": {
                "name": "PLAYER1",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/face-2.png"), (self.cell_size, self.cell_size))
                },
            "PLAYER2": {
                "name": "PLAYER2",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/face-3.png"), (self.cell_size, self.cell_size))
                },
            "PLAYER3": {
                "name": "PLAYER3",
                "color":pygame.transform.scale(pygame.image.load(self.PATH + "/assets/face-4.png"), (self.cell_size, self.cell_size))
                },
        }
        
        self.valid_blocks = [1, 7, 8, 9, 10, 11, 12, 13, 14]
        self.possible_agents = ["player_" + str(r) for r in range(4)]
        self.grid = np.array([
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [3,3,3,3,3,3,3,3,3,3,3,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [4,4,4,4,4,4,4,4,4,4,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [5,5,5,5,5,5,5,5,5,5,5,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [6,6,6,6,6,6,6,6,6,6,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [8,8,8,8,8,8,8,8,8,8,8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [9,9,9,9,9,9,9,9,9,9,9,1,1,1,1,1,1,1,1,1,1,1,5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [10,10,10,10,10,10,10,10,10,10,10,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [11,11,11,11,11,11,11,11,11,11,11,1,1,1,1,1,1,1,1,1,1,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [12,12,12,12,12,12,12,12,12,12,12,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [13,13,13,13,13,13,13,13,13,13,13,1,1,1,1,1,1,1,1,1,1,1,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [14,14,14,14,14,14,14,14,14,14,14,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
        ])

        self.display = pygame.display.set_mode((self.cell_size*self.width, self.cell_size*self.heigth))
        pygame.display.set_caption("Unrailed")
        
        self.agents_pos = np.asarray([
            #[y, x, deg, item, #items]
            [9, 16, 0, 0],
            [10, 16, 0, 0],
            [11, 16, 0, 0],
            [12, 16, 0, 0],
        ])
        
        # optional: a mapping between agent name and ID
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )
        self.render_mode = render_mode

    # Observation space should be defined here.
    # lru_cache allows observation and action spaces to be memoized, reducing clock cycles required to get each agent's space.
    # If your spaces change over time, remove this line (disable caching).
    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
        return Discrete(4)

    # Action space should be defined here.
    # If your spaces change over time, remove this line (disable caching).
    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return Discrete(3)

    def render(self):
        """
        Renders the environment. In human mode, it can print to terminal, open
        up a graphical window, or open up some other display that a human can see and understand.
        """
        if self.render_mode is None:
            gymnasium.logger.warn(
                "You are calling render method without specifying any render mode."
            )
            return
        
        self.display.fill((156, 215, 86))
        for i in range(self.heigth):
            for j in range(self.width):
                if self.grid[i, j] == 0:
                    pygame.draw.rect(self.display, (0, 0, 0), (j * self.cell_size, i * self.cell_size, self.cell_size, self.cell_size))
                else:
                    self.display.blit(self.population[self.grid[i, j]]["color"], [self.cell_size*j, self.cell_size*i])
                    
                # Draw players
                if [i, j] == self.agents_pos[0][:2]:
                    self.display.blit(pygame.transform.rotate(self.population["PLAYER0"]["color"], self.agents_pos[0][2]), [self.cell_size*j, self.cell_size*i])
                elif [i, j] == self.agents_pos[1][:2]:
                    self.display.blit(pygame.transform.rotate(self.population["PLAYER1"]["color"], self.agents_pos[1][2]), [self.cell_size*j, self.cell_size*i])
                elif [i, j] == self.agents_pos[2][:2]:
                    self.display.blit(pygame.transform.rotate(self.population["PLAYER2"]["color"], self.agents_pos[2][2]), [self.cell_size*j, self.cell_size*i])
                elif [i, j] == self.agents_pos[3][:2]:
                    self.display.blit(pygame.transform.rotate(self.population["PLAYER3"]["color"], self.agents_pos[3][2]), [self.cell_size*j, self.cell_size*i])
        pygame.display.flip()


        if len(self.agents) == 2:
            string = "Current state: Agent1: {} , Agent2: {}".format(
                MOVES[self.state[self.agents[0]]], MOVES[self.state[self.agents[1]]]
            )
        else:
            string = "Game over"
        print(string)
        
    def plyer_move(self, action, player):
        if action == 0:
            self.agents_pos[player][2] = 90
            if self.agents_pos[player][0] == 0:
                self.agents_pos[player][0] = max(0, self.agents_pos[player][0] - 1)
            elif self.grid[self.agents_pos[player][0]-1, self.agents_pos[player][1]] in self.valid_blocks:
                self.agents_pos[player][0] = self.agents_pos[player][0] - 1
        if action == 1:
            self.agents_pos[player][2] = 270
            if self.agents_pos[player][0] == self.heigth-1:
                self.agents_pos[player][0] = min(self.heigth - 1, self.agents_pos[player][0] + 1)  
            elif self.grid[self.agents_pos[player][0]+1, self.agents_pos[player][1]] in self.valid_blocks:
                self.agents_pos[player][0] = self.agents_pos[player][0] + 1
        if action == 2:
            self.agents_pos[player][2] = 180
            if self.agents_pos[player][1] == 0:
                self.agents_pos[player][1] = max(0, self.agents_pos[player][1] - 1)
            elif self.grid[self.agents_pos[player][0], self.agents_pos[player][1]-1] in self.valid_blocks:
                self.agents_pos[player][1] = self.agents_pos[player][1] - 1
        if action == 3:
            self.agents_pos[player][2] = 0
            if self.agents_pos[player][1] == self.width-1:
                self.agents_pos[player][1] = min(self.width - 1, self.agents_pos[player][1] + 1)
            elif self.grid[self.agents_pos[player][0], self.agents_pos[player][1]+1] in self.valid_blocks:
                self.agents_pos[player][1] = self.agents_pos[player][1] + 1
            
        
        return self.agents_pos, 0, False, {}


    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, options=None):
        """
        Reset needs to initialize the `agents` attribute and must set up the
        environment so that render(), and step() can be called without issues.
        Here it initializes the `num_moves` variable which counts the number of
        hands that are played.
        Returns the observations for each agent
        """
        self.agents = self.possible_agents[:]
        self.num_moves = 0
        observations = {agent: NONE for agent in self.agents}
        infos = {agent: {} for agent in self.agents}
        self.state = observations

        return observations, infos

    def step(self, actions):
        """
        step(action) takes in an action for each agent and should return the
        - observations
        - rewards
        - terminations
        - truncations
        - infos
        dicts where each dict looks like {agent_1: item_1, agent_2: item_2}
        """
        # If a user passes in actions with no agents, then just return empty observations, etc.
        if not actions:
            self.agents = []
            return {}, {}, {}, {}, {}

        # rewards for all agents are placed in the rewards dictionary to be returned
        rewards = {}
        rewards[self.agents[0]], rewards[self.agents[1]] = REWARD_MAP[
            (actions[self.agents[0]], actions[self.agents[1]])
        ]

        terminations = {agent: False for agent in self.agents}

        self.num_moves += 1
        env_truncation = self.num_moves >= NUM_ITERS
        truncations = {agent: env_truncation for agent in self.agents}

        # current observation is just the other player's most recent action
        observations = {
            self.agents[i]: int(actions[self.agents[1 - i]])
            for i in range(len(self.agents))
        }
        self.state = observations

        # typically there won't be any information in the infos, but there must
        # still be an entry for each agent
        infos = {agent: {} for agent in self.agents}

        if env_truncation:
            self.agents = []

        if self.render_mode == "human":
            self.render()
        return observations, rewards, terminations, truncations, infos

In [4]:
env = UnrailedEnv()

# Bucle principal
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                _, _, done, _ = env.step(0, 0)
            elif event.key == pygame.K_DOWN:
                _, _, done, _ = env.step(1, 0)
            elif event.key == pygame.K_LEFT:
                _, _, done, _ = env.step(2, 0)
            elif event.key == pygame.K_RIGHT:
                _, _, done, _ = env.step(3, 0)
            if done:
                env.reset()
    
    env.render()
    
pygame.quit()

NameError: name '__file__' is not defined

In [7]:
actions = [0, 1, 2, 3, 4, 5]

In [5]:
possible_agents = ["player_" + str(r) for r in range(4)]
agents = possible_agents[:]

In [6]:
agents

['player_0', 'player_1', 'player_2', 'player_3']

In [ ]:
0: {
    "name": "PLAYER",
    },
1: {
    "name": "GRASS",
    },
2: {
    "name": "TREE",
    },
3: {
    "name": "ROCK",
    },
4: {
    "name": "SOLID",
    },
5: {
    "name": "WATER",
    },
6: {
    "name": "HOUSE",
    },
7: {
    "name": "BRIDGE",
    },
8: {
    "name": "RAIL",
    },
9:{
    "name": "WOOD",
},
10:{
    "name": "IRON",
},
11:{
    "name": "AXE",
},
12:{
    "name": "PICKAXE",
},
13:{
    "name": "BUCKET",
},
14:{
    "name": "EMPTY_BUCKET",
}
15:{
    "name": "ENGINE",
},
16:{
    "name": "CRAFTING",
},
17:{
    "name": "STORAGE",
},
18:{
    "name": "TANK",
},


In [2]:
import numpy as np

In [22]:
grid = np.array([
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [3,3,3,3,3,3,3,3,3,3,3,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [4,4,4,4,4,4,4,4,4,4,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [5,5,5,5,5,5,5,5,5,5,5,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [6,6,6,6,6,6,6,6,6,6,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [8,8,8,8,8,8,8,8,8,8,8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [9,9,9,9,9,9,9,9,9,9,9,1,1,1,1,1,1,1,1,1,1,1,5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [10,10,10,10,10,10,10,10,10,10,10,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [11,11,11,11,11,11,11,11,11,11,11,1,1,1,1,1,1,1,1,1,1,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [12,12,12,12,12,12,12,12,12,12,12,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [13,13,13,13,13,13,13,13,13,13,13,1,1,1,1,1,1,1,1,1,1,1,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [14,14,14,14,14,14,14,14,14,14,14,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
])

In [30]:
def getAgentView(x, y):
    fila_inicio = max(0, x - 1)
    fila_fin = min(grid.shape[0], x + 2)
    col_inicio = max(0, y - 1)
    col_fin = min(grid.shape[1], y + 2)

    # Crear una matriz de -1 con las dimensiones de la ventana
    ventana = np.full((3, 3), -1)

    # Obtener las coordenadas válidas en la ventana
    ventana_fila_inicio = max(0, 1 - x)
    ventana_fila_fin = ventana_fila_inicio + (fila_fin - fila_inicio)
    ventana_col_inicio = max(0, 1 - y)
    ventana_col_fin = ventana_col_inicio + (col_fin - col_inicio)

    # Actualizar la parte válida de la ventana con los valores de la matriz
    ventana[ventana_fila_inicio:ventana_fila_fin, ventana_col_inicio:ventana_col_fin] = grid[fila_inicio:fila_fin, col_inicio:col_fin]

    return ventana

In [32]:
getAgentView(0, 0)

array([[-1, -1, -1],
       [-1,  1,  1],
       [-1,  0,  0]])

In [21]:
grid[15][22]

8

In [20]:
def encontrar_posicion_mas_cercana_con_valor(arreglo_2d, valor_objetivo, posicion_objetivo):
    posiciones = np.argwhere(arreglo_2d == valor_objetivo)  # Obtener las posiciones donde el valor sea igual a n
    distancias = np.linalg.norm(posiciones - posicion_objetivo, axis=1)  # Calcular la distancia a la posición objetivo
    indice_posicion_mas_cercana = np.argmin(distancias)  # Encontrar el índice de la posición con la distancia mínima
    if indice_posicion_mas_cercana is not None:
        posicion_mas_cercana = posiciones[indice_posicion_mas_cercana]  # Obtener la posición con la distancia mínima
        return tuple(posicion_mas_cercana)
    else:
        return None  # No se encontraron posiciones con el valor objetivo

# Ejemplo de uso:
arreglo_2d = [[0, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 0, 0]]

valor_objetivo = 1
posicion_objetivo = (18, 29)
posicion_cercana = encontrar_posicion_mas_cercana_con_valor(grid, 8, posicion_objetivo)
posicion_cercana

(15, 22)

In [36]:
arreglo_2d = np.array([[0, 0, 0],
            [0, 1, 0],
            [0, 0, 0]])

arreglo_2d[:,:2]

array([[0, 0],
       [0, 1],
       [0, 0]])

In [ ]:
self.rewards_values = {
    "pick_item": 10,
    "break_block": 10,
    "fill_bucket": 10,
    "pick_same_item": -100,
    "pick_wrong_item": -100,
    "complete_task": 1000,
}
self.valid_items = {
    0: [9, 11],
    1: [10, 12],
    2: [8],
    3: [13, 14]
}
self.valid_actions_blocks = {
    0: [2],
    1: [3],
    2: [8],
    3: [5]
}
self.valid_action_targets = {
    0: [2],
    1: [3],
    2: [16],
    3: [5],
}
self.drops = {
    2: 9,
    3: 10,
}
self.valid_tools_targets = {
    11: [2],
    12: [3],
    13: [16],
    14: [5],
}


In [ ]:
self.observations = {
            self.agents[i]: {
            "vision"
            "position"
            "orientation"
            "item"
            "wood_item"
            "rock_item"
            "pickaxe"
            "axe": 
            "bucket"
            "rock_block"
            "tree_block"
            "water"
            "refri"
            "deposit"
            "factory"
            "riel"
            "house"
            "TASK"
        } for i in range(len(self.agents)) 
        }

In [ ]:
 self.actions = [0, 1, 2, 3, 4, 5]
self.breakeble = [2, 3]
self.pickeable = [8, 9, 10, 11, 12, 13, 14]
self.valid_blocks = [1, 7, 8, 9, 10, 11, 12, 13, 14]

In [ ]:
                            #arriba, derecha, abajo, izquierda   
        self.orientations = [0, 1, 2, 3]

In [11]:
grid = [[0, 1],
        [1, 0]]
grid[1-1]

[0, 1]

In [3]:
env = parallel_env(render_mode="human")
observations, infos = env.reset()

while env.agents:
    # this is where you would insert your policy
    actions = {agent: env.action_space(agent).sample() for agent in env.agents}

    observations, rewards, terminations, truncations, infos = env.step(actions)
env.close()

Current state: Agent1: SCISSORS , Agent2: ROCK
Current state: Agent1: ROCK , Agent2: SCISSORS
Current state: Agent1: SCISSORS , Agent2: ROCK
Current state: Agent1: PAPER , Agent2: PAPER
Current state: Agent1: ROCK , Agent2: ROCK
Current state: Agent1: PAPER , Agent2: SCISSORS
Current state: Agent1: ROCK , Agent2: ROCK
Current state: Agent1: SCISSORS , Agent2: SCISSORS
Current state: Agent1: ROCK , Agent2: PAPER
Current state: Agent1: PAPER , Agent2: ROCK
Current state: Agent1: PAPER , Agent2: PAPER
Current state: Agent1: SCISSORS , Agent2: ROCK
Current state: Agent1: ROCK , Agent2: ROCK
Current state: Agent1: ROCK , Agent2: SCISSORS
Current state: Agent1: PAPER , Agent2: SCISSORS
Current state: Agent1: ROCK , Agent2: SCISSORS
Current state: Agent1: PAPER , Agent2: PAPER
Current state: Agent1: ROCK , Agent2: PAPER
Current state: Agent1: PAPER , Agent2: SCISSORS
Current state: Agent1: SCISSORS , Agent2: ROCK
Current state: Agent1: SCISSORS , Agent2: ROCK
Current state: Agent1: SCISSORS ,